# Estimando distribuciones (parte 1)
El objetivo de esta notebook es explorar una primera manera de aproximar $p(y|x)$ y $p(x|y)$ en un set de datos tabular. En este set de datos $x$ tiene valores discretos, $x\in\mathbb{D}^k$, y el target $y$ es un booleano, $y\in\{0,1\}$.

## Imports

In [242]:
import numpy as np
import pandas as pd
import random

## Cargamos los datos

In [243]:
df = pd.read_csv('./tennis.csv', delimiter=',', header=0)
df

,Day,Outlook,Temp,Humidity,Wind,Tennis
0,D1,Sunny,Hot,High,Weak,No
1,D2,Sunny,Hot,High,Strong,No
2,D3,Overcast,Hot,High,Weak,Yes
3,D4,Rain,Mild,High,Weak,Yes
4,D5,Rain,Cool,Normal,Weak,Yes
5,D6,Rain,Cool,Normal,Strong,No
6,D7,Overcast,Cool,Normal,Strong,Yes
7,D8,Sunny,Mild,High,Weak,No
8,D9,Sunny,Cool,Normal,Weak,Yes
9,D10,Rain,Mild,Normal,Weak,Yes


### Eliminamos la columna Day 

In [244]:
df = df.drop('Day', axis=1)
df

,Outlook,Temp,Humidity,Wind,Tennis
0,Sunny,Hot,High,Weak,No
1,Sunny,Hot,High,Strong,No
2,Overcast,Hot,High,Weak,Yes
3,Rain,Mild,High,Weak,Yes
4,Rain,Cool,Normal,Weak,Yes
5,Rain,Cool,Normal,Strong,No
6,Overcast,Cool,Normal,Strong,Yes
7,Sunny,Mild,High,Weak,No
8,Sunny,Cool,Normal,Weak,Yes
9,Rain,Mild,Normal,Weak,Yes


In [245]:
X_names = df.columns.to_list()[:-1]
X_names

['Outlook', 'Temp', 'Humidity', 'Wind']

Guardamos en la variable $X$ todas las features del dataset.

In [246]:
X = df.iloc[:,0:-1]
X

,Outlook,Temp,Humidity,Wind
0,Sunny,Hot,High,Weak
1,Sunny,Hot,High,Strong
2,Overcast,Hot,High,Weak
3,Rain,Mild,High,Weak
4,Rain,Cool,Normal,Weak
5,Rain,Cool,Normal,Strong
6,Overcast,Cool,Normal,Strong
7,Sunny,Mild,High,Weak
8,Sunny,Cool,Normal,Weak
9,Rain,Mild,Normal,Weak


In [247]:
Y_name = df.columns.to_list()[-1]
Y_name

'Tennis'

Guardamos en $Y$ el objetivo

In [248]:
Y = df.iloc[:,-1]
Y

0      No
1      No
2     Yes
3     Yes
4     Yes
5      No
6     Yes
7      No
8     Yes
9     Yes
10    Yes
11    Yes
12    Yes
13     No
Name: Tennis, dtype: object

## Construimos una tabla de observaciones

En este paso vamos a crear una tabla de observaciones. Esta tabla tiene que contener la frecuencia de cada observación. Para este ejemplo tomaremos a $x$ como **Outlook**.

Calcule las dimensiones de la tabla

In [249]:
# Columna observacion x
df_obs = df[X_names[0]]

# Cantidad total de elementos
N = df_obs.size

# Elementos únicos de la clase Outlook
xvalues = df_obs.unique()
dimx = len(xvalues)

# Elementos únicos del objetivo
yvalues = Y.unique()
dimy = len(yvalues)

print(f'Cantidad total de elementos: {N}')
print(f'Elementos únicos de la clase Outlook: {xvalues}')
print(f'Elementos únicos del objetivo: {yvalues}')

Cantidad total de elementos: 14
Elementos únicos de la clase Outlook: ['Sunny' 'Overcast' 'Rain']
Elementos únicos del objetivo: ['No' 'Yes']


Calculamos la tabla de frecuencia.

In [250]:
obs = pd.DataFrame(0, columns=yvalues, index=xvalues, dtype=float)

## Llene la tabla de observaciones
for iteration in range(N):
    x_iter = df_obs[iteration]
    y_iter = Y[iteration]
    obs.loc[x_iter,y_iter] += 1
    
obs

,No,Yes
Sunny,3.0,2.0
Overcast,0.0,4.0
Rain,2.0,3.0


## Aproximación de la distribución conjunta $p(x,y)$

Tome a $x$ como Outlook y aproxime la distribución conjunta utilizando la tabla de observaciones. 

In [251]:
joint_x_y = obs / N
joint_x_y

,No,Yes
Sunny,0.214286,0.142857
Overcast,0.000000,0.285714
Rain,0.142857,0.214286


1. ¿Qué significa el valor calculado en los índices "Sunny", "Yes"?

2. ¿Justifique el resultado de la pareja "Overcast", "No"?

#### 1. Es la probabilidad de que se juegue al tenis y el día esté soleado, según nuestra tabla de datos.

#### 2. Esta probabilidad conjunta es correcta, ya que según nuestra tabla, siempre que estuvo nublado se jugó al tenis. Por lo tanto, la probabilidad conjunta de que esté nublado y no se juegue es 0.

## Aproximamos $p(y|x)$

Tome a $x$ como **Outlook** y estime la probabilidad condicional de $y$ dado $x$. Luego realice una muestra de 10 valores de $y$ dado $x = Sunny$.

Calculamos la cantidad de entradas por cada valor distinto de $x$.

In [252]:
m = obs.sum(axis=1)
obs['m'] = m
m

Sunny       5.0
Overcast    4.0
Rain        5.0
dtype: float64

Calculamos la cantidad de entradas por cada valor distinto de $y$.

In [253]:
l = obs.sum(axis=0)
obs.loc['l'] = l
l

No      5.0
Yes     9.0
m      14.0
dtype: float64

In [254]:
obs

,No,Yes,m
Sunny,3.0,2.0,5.0
Overcast,0.0,4.0,4.0
Rain,2.0,3.0,5.0
l,5.0,9.0,14.0


Calcule la probabilidad condicional de $y$ dado $x$.

In [255]:
p_y_x = pd.DataFrame(0, columns=yvalues, index=xvalues, dtype=float)

## Llene la tabla de probabilidades condicionales p(y|x)

for x_iter in xvalues:
    for y_iter in yvalues:
        px = obs.loc[x_iter,'m'] / obs.loc['l','m']
        pxy = obs.loc[x_iter,y_iter] / obs.loc['l','m']
        p_y_x.loc[x_iter,y_iter] = pxy / px

p_y_x

,No,Yes
Sunny,0.6,0.4
Overcast,0.0,1.0
Rain,0.4,0.6


3. ¿La suma de cada fila siempre tiene que dar 1? ¿Por qué?

4. ¿Y si la suma de las columnas?

#### 3. Cada fila representa el espacio de resultados para la variable objetivo. Por lo tanto, la distribucion tiene que sumar 1. La variable puede ser "Yes" o "No", son excluyentes, por lo tanto sus probabilidades suman 1.

#### 4. No tienen porque, por lo explicado antes. Las filas entre si corresponden a espacios distintos, por lo tanto no hay vinculo entre la probabilidad de una misma columna.

Realice 10 muestras de $y$ dado $x = Sunny$.

Puede utilizar la función random.choice de numpy. https://numpy.org/doc/stable/reference/random/generated/numpy.random.choice.html

In [256]:
df_sampled = df[df['Outlook'] == 'Sunny'][Y_name]

df_sampled

0      No
1      No
7      No
8     Yes
10    Yes
Name: Tennis, dtype: object

In [257]:
sampled_values = df_sampled.sample(n=10, replace=True).to_list()

sampled_values

['No', 'Yes', 'No', 'No', 'No', 'Yes', 'No', 'No', 'Yes', 'No']

5. ¿Qué pasaría si utilizamos $x = Overcast$ en vez de $x = Sunny$? ¿Tiene sentido que pase esto? ¿Por qué?

#### 5. Obtendría 10 muestras con la variable objetivo "No". Esto tiene sentido, ya que la distribución en el DataFrame solo incluye "Yes", porque en nuestra muestra de datos no hay ocurrencias en conjunto de "Overcast" y "Yes".

## Aproximamos $p(x|y)$
Tome a $x$ como Outlook y estime la probabilidad condicional de $x$ dado $y$ basandose en la tabla de observaciones. Luego realice 10 muestras de $x$ dado $y = Yes$.

$p(x|y)$

In [258]:
obs 

,No,Yes,m
Sunny,3.0,2.0,5.0
Overcast,0.0,4.0,4.0
Rain,2.0,3.0,5.0
l,5.0,9.0,14.0


In [259]:
p_x_y = pd.DataFrame(0, columns=yvalues, index=xvalues, dtype=float)

## Llene la tabla de probabilidades condicionales p(x|y)

for x_iter in xvalues:
    for y_iter in yvalues:
        py = obs.loc['l',y_iter] / obs.loc['l','m']
        pxy = obs.loc[x_iter,y_iter] / obs.loc['l','m']
        p_x_y.loc[x_iter,y_iter] = pxy / py

p_x_y

,No,Yes
Sunny,0.6,0.222222
Overcast,0.0,0.444444
Rain,0.4,0.333333


Muestreo

In [260]:
df_filtered = df[df['Tennis'] == 'Yes'][X_names[0]]

df_filtered

2     Overcast
3         Rain
4         Rain
6     Overcast
8        Sunny
9         Rain
10       Sunny
11    Overcast
12    Overcast
Name: Outlook, dtype: object

In [261]:
sampled_values = df_filtered.sample(n=10, replace=True).to_list()

sampled_values

['Rain',
 'Overcast',
 'Overcast',
 'Rain',
 'Overcast',
 'Overcast',
 'Overcast',
 'Rain',
 'Overcast',
 'Sunny']

## Aproxime $p(y,o,h,w,t)$

Aproxime la proabilidad conjunta del Tennis (y), Outlook (o), Humidity (h), Wind (w), Temp (t).

Recuerde que $p(y,o,h,w,t)$ = $p(y)$.$p(o|y)$.$p(h|y,o)$.$p(w|y,o,h)$.$p(t|y,o,h,w)$

Calcule P(y)

In [262]:
# P(Y)
p_y = df.Tennis.value_counts(normalize=True)
p_y

Tennis
Yes    0.642857
No     0.357143
Name: proportion, dtype: float64

Calcule P(o|y)

In [263]:
p_o_y = pd.crosstab(index=df['Outlook'], columns=df['Tennis'], normalize='columns')

p_o_y = p_o_y.fillna(0)

p_o_y

Tennis,No,Yes
Outlook,,
Overcast,0.0,0.444444
Rain,0.4,0.333333
Sunny,0.6,0.222222


Calcule P(h|y,o)

Recomendamos usar la función *crosstab* de pandas. En este link pueden encontrar un ejemplo de su uso: https://www.geeksforgeeks.org/pandas-crosstab-function-in-python/

In [264]:
# Calcule la tabla de frecuencia
p_h_yo = pd.crosstab(index=df['Humidity'], columns=[df['Tennis'],df['Outlook']], normalize='columns')

# Luego se divide cada fila por la suma de sus elementos, puede usar las funciones div y sum de pandas

# No se oliden de llenar los valores NaN!!!
p_h_yo = p_h_yo.fillna(0)

# Cambiamos los nombres de los indices (si es que lo precisa)

p_h_yo

Tennis     No            Yes                
Outlook  Rain Sunny Overcast      Rain Sunny
Humidity                                    
High      0.5   1.0      0.5  0.333333   0.0
Normal    0.5   0.0      0.5  0.666667   1.0

Calcule P(w|y,o,h)

In [265]:
p_w_yoh = pd.crosstab(index=df['Wind'], columns=[df['Tennis'],df['Outlook'],df['Humidity']], normalize='columns')

p_w_yoh = p_w_yoh.fillna(0)

p_w_yoh

Tennis     No                       Yes                          
Outlook  Rain            Sunny Overcast        Rain         Sunny
Humidity High Normal      High     High Normal High Normal Normal
Wind                                                             
Strong    1.0    1.0  0.333333      0.5    0.5  0.0    0.0    0.5
Weak      0.0    0.0  0.666667      0.5    0.5  1.0    1.0    0.5

Calcule P(t|y,o,h,w)

In [266]:
p_t_yohw = pd.crosstab(index=df['Temp'], columns=[df['Tennis'],df['Outlook'],df['Humidity'],df['Wind']], normalize='columns')

p_t_yohw = p_t_yohw.fillna(0)

p_t_yohw

Tennis       No                         Yes                               \
Outlook    Rain         Sunny      Overcast                  Rain          
Humidity   High Normal   High          High      Normal      High Normal   
Wind     Strong Strong Strong Weak   Strong Weak Strong Weak Weak   Weak   
Temp                                                                       
Cool        0.0    1.0    0.0  0.0      0.0  0.0    1.0  0.0  0.0    0.5   
Hot         0.0    0.0    1.0  0.5      0.0  1.0    0.0  1.0  0.0    0.0   
Mild        1.0    0.0    0.0  0.5      1.0  0.0    0.0  0.0  1.0    0.5   

Tennis                
Outlook   Sunny       
Humidity Normal       
Wind     Strong Weak  
Temp                  
Cool        0.0  1.0  
Hot         0.0  0.0  
Mild        1.0  0.0

Calcule P(y,o,h,w,t)

In [267]:
# Definimos una función que nos calcula la probabilidad conjunta usando la regla del producto.
def calculate_prob(y,o,h,w,t):
    "Calculate the probability of occurrence of a row"
    p = p_y[y] * p_o_y.loc[o,y] * p_h_yo.loc[h,(y,o)] * p_w_yoh.loc[w,(y,o,h)] * p_t_yohw.loc[t,(y,o,h,w)]
    
    return p

In [268]:
prob = calculate_prob('Yes','Sunny','Normal','Weak','Cool')

print(f'P(Yes|Sunny,Normal,Weak,Cool) = {prob}')
print(f"Cantidad de observaciones: {prob*N}")

P(Yes|Sunny,Normal,Weak,Cool) = 0.07142857142857142
Cantidad de observaciones: 1.0


Definimos el muestreo de datos completos.

Primero generamos un *y* utilizando *p(y)* y luego seguimos con las probabilidades condicionales.

In [ ]:
def sample_from_y(y):
    "Generates a sample of weather conditions based on a specific y"
    sampled_o = np.random.choice(p_o_y.index, p=p_o_y[y])
    sampled_h = np.random.choice(p_h_yo.index, p=p_h_yo[(y,sampled_o)])
    sampled_w = np.random.choice(p_w_yoh.index, p=p_w_yoh[(y,sampled_o,sampled_h)])
    sampled_t = np.random.choice(p_t_yohw.index, p=p_t_yohw[(y,sampled_o,sampled_h,sampled_w)])

    return [y, sampled_o, sampled_h, sampled_w, sampled_t]
    
def sample():
    "Generates a sample of weather conditions based on a random y"
    y = np.random.choice(p_y.index, p=p_y)
    # En caso de que se quiera que sea completamente random. Se puede descomentar la siguiente línea.
    # y = random.choice(p_y.index)

    return sample_from_y(y)

In [270]:
samples = np.array([sample() for _ in range(len(df))])
new_df = pd.DataFrame(samples, columns=['Tennis', 'Outlook', 'Humidity', 'Wind', 'Temp'])
new_df

,Tennis,Outlook,Humidity,Wind,Temp
0,No,Rain,Normal,Strong,Cool
1,Yes,Rain,High,Weak,Mild
2,No,Sunny,High,Strong,Hot
3,Yes,Overcast,High,Weak,Hot
4,No,Sunny,High,Strong,Hot
5,Yes,Rain,Normal,Weak,Cool
6,No,Sunny,High,Strong,Hot
7,No,Sunny,High,Weak,Mild
8,Yes,Sunny,Normal,Strong,Mild
9,Yes,Overcast,High,Strong,Mild
